In [1]:
class Caminhao:
    def __init__(self, items = 0, doca = -1, horaEntrada = -1):
        self.NAO_HOUVE_DELIBERACAO = -1
        self.__doca = doca
        self.__horaEntrada = horaEntrada
        self.__horaDescarregando = self.NAO_HOUVE_DELIBERACAO
        self.__horaDeSaida = self.NAO_HOUVE_DELIBERACAO
        self.__items = items
    
    def getDoca(self):
        if self.__doca == self.NAO_HOUVE_DELIBERACAO:
            return 'ESPERANDO DECISAO'
        else:
            return self.__doca

    def setDoca(self, doca, horaEntrada):
        self.__doca = doca
        self.__horaEntrada = horaEntrada

    def comecaDescarregar(self, hora):
        self.__horaDescarregando = hora
        
    def getHoraDescarregando(self):
        return self.__horaDescarregando
        
    def getTempoDescarregando(self, hora):
        if self.__horaDescarregando == self.NAO_HOUVE_DELIBERACAO:
            return self.NAO_HOUVE_DELIBERACAO
        else:
            return hora - self.__horaDescarregando
        
    def getTempoNaFila(self, hora):
        if self.__doca == self.NAO_HOUVE_DELIBERACAO:
            return self.NAO_HOUVE_DELIBERACAO
        else:
            return hora - self.__horaEntrada            
    
    def setHoraDeSaida(self, hora):
        self.__horaDeSaida = hora
    
    def getHoraDeSaida(self):
        return self.__horaDeSaida
    
    def getHoraEntrada(self):
        return self.__horaEntrada
    
    def tempoTotalDescarregando(self):
        return self.getTempoDescarregando(self.__horaDeSaida)
    
    def tempoTotalNaFila(self):
        return self.getTempoNaFila(self.__horaDeSaida)
    
    @property
    def items(self):
        return self.__items
    
    @items.setter
    def items(self, items):
        self.__items = items
        
    
    

In [2]:
class Docas:
    def __init__(self, nDocas = 4):
        #self.TEMPO_ENTRE_CAMINHOES = 0 # ativo como o delay do update
        self.__NAO_HOUVE_DELIBERACAO = -1
        self.__historicoDeCaminhoes = []
        self.__caminhoesDasDocas = {}
        for i in range(nDocas):
            self.__caminhoesDasDocas[i] = []

    def addCaminhao(self, caminhao):
        iDoca = caminhao.getDoca()
        self._checkDoca(iDoca)
        self.__caminhoesDasDocas[iDoca].append(caminhao)
    
    def updateDescarregamento(self, hora):
        for iDoca in self.__caminhoesDasDocas:
            if len(self.__caminhoesDasDocas[iDoca]) == 0:
                continue
            else:
                caminhaoI = self.__caminhoesDasDocas[iDoca][0]
                tDesc = caminhaoI.getTempoDescarregando(hora)
                if tDesc == self.__NAO_HOUVE_DELIBERACAO:
                    caminhaoI.comecaDescarregar(hora)
                elif terminouDeDescarregar(caminhaoI,hora):
                    caminhaoI.setHoraDeSaida(hora)
                    self.__historicoDeCaminhoes.append(caminhaoI)
                    self.__caminhoesDasDocas[iDoca].pop(0)

    def getHistorico(self):
        return self.__historicoDeCaminhoes
                    
    def getNumeroDocas(self):
        return len(self.__caminhoesDasDocas)

    def getDocaComMenosItems(self):
        minNumero = float('inf')
        iDocaMin = -1
        for iDoca in self.__caminhoesDasDocas:
            nItems = self.getNumeroDeItemsNaDoca(iDoca)
            if nItems < minNumero:
                iDocaMin = iDoca
                minNumero = nItems
        return iDocaMin

    def getNumeroDeItemsNaDoca(self, iDoca):
        self._checkDoca(iDoca)
        itemsDoca = 0
        for caminhao in self.__caminhoesDasDocas[iDoca]:
            itemsDoca += caminhao.items
        return itemsDoca

    def getDocaComMenosCaminhoes(self):
        minNumero = float('inf')
        iDocaMin = -1
        for iDoca in self.__caminhoesDasDocas:
            nCaminhoes = self.getNumeroCaminhoesNaDoca(iDoca)
            if nCaminhoes < minNumero:
                iDocaMin = iDoca
                minNumero = nCaminhoes
        return iDocaMin
                
    def getNumeroCaminhoesNaDoca(self, iDoca):
        self._checkDoca(iDoca)
        return len(self.__caminhoesDasDocas[iDoca])
    
    def getTempoDescarregandoNaDoca(self, iDoca, hora):
        self._checkDoca(iDoca)
        if len(self.__caminhoesDasDocas[iDoca]) == 0:
            return -1
        else:
            return self.__caminhoesDasDocas[iDoca][0].getTempoDescarregando(hora)

        
    def getNumeroItemsDeCadaCaminhao(self, iDoca):
        itemsLista = []
        if len(self.__caminhoesDasDocas[iDoca]) == 0:
            return itemsLista
        for caminhaoI in self.__caminhoesDasDocas[iDoca]:
            itemsLista.append(caminhaoI.items)
        return itemsLista
        
        
    def _checkDoca(self, iDoca):
        if iDoca not in self.__caminhoesDasDocas:
            raise Exception('A doca ' + str(iDoca) + ' nao foi encontrada')
        
            

In [3]:
def terminouDeDescarregar(caminhao, hora):
    return caminhao.getTempoDescarregando(hora) > caminhao.items
 
def calculateTempoTotalMedio(historico):
    tempoTotalMedio = 0
    for caminhaoI in historico:
        tempoTotalMedio += caminhaoI.tempoTotalNaFila()
    tempoTotalMedio /= len(historico)
    return tempoTotalMedio    

def verificaFinal(docas, hora):
    if hora%20 == 0:
        if VERBOSE:
            print('Hora:  ', hora)
            print('Doca   :  Numero de caminhoes', hora)
        numeroCaminhoesAtivos = 0
        for i in range(docas.getNumeroDocas()):
            nCaminhoes = docas.getNumeroCaminhoesNaDoca(i)
            numeroCaminhoesAtivos += nCaminhoes
            if VERBOSE:
                print(' ',i,'   :      ',nCaminhoes)
        return numeroCaminhoesAtivos == 0 and hora > TEMPO_LIMITE
    else:
        return False
    
def adicionarCaminhao(hora, docas):
    if hora%np.random.randint(2,10) == 0 and hora < TEMPO_LIMITE:
        iDoca = calculeMelhorDoca(docas, hora)
        #iDoca = docas.getDocaComMenosItems()
        #iDoca = docas.getDocaComMenosCaminhoes()
        #iDoca = np.random.choice(docas.getNumeroDocas())                   # politica de escolha
        #items = ITEMS                                      
        items = 5 * np.random.randint(2,20)
        docas.addCaminhao(Caminhao(items, iDoca, hora))
        
def simuleUmDia():
    docas = Docas()
    for hora in range(10000):
        adicionarCaminhao(hora, docas)
        docas.updateDescarregamento(hora)
        if verificaFinal(docas, hora):
            break
    tempoMedio = calculateTempoTotalMedio(docas.getHistorico())
    return [tempoMedio, docas]
    

In [4]:
import numpy as np
class PredicaoDoTempoNaDoca:
    def __init__(self):
        self.__TAXA_APRENDIZADO = 1e-4
        self.__nParameters = 3
        self.resetParameters()
        pass
    
    def resetParameters(self):
        self.__parameters = np.random.random(self.__nParameters)

    def getParameters(self):
        return self.__parameters
        
    def transformX(self, x):
        return np.array([1,x[0],x[1]])
        
    def previsao(self, x):
        x = self.transformX(x)
        return self.__parameters.dot(x) # a + b*items no caminhao + c*numero doca
    
    def atualizandoParametros(self, x, valorReal):
        prev = self.previsao(x)
        gradiente = -self.__TAXA_APRENDIZADO * (prev-valorReal) * self.transformX(x)
        self.__parameters += gradiente


In [38]:
TEMPO_LIMITE = 80
VERBOSE = False
ITEMS = 15
med = 0
for i in range(1000):
    tempoMedio, _ = simuleUmDia()
    med += tempoMedio
print(med/1000) #824 aleatorio, 719 - numero, 694 - items

123.62684668318064


In [4278]:
modeloPrevisao = PredicaoDoTempoNaDoca()

In [4279]:
modeloPrevisao.previsao([45,0])

17.410582425934805

In [4329]:
modeloPrevisao.atualizandoParametros([45,0],50)
modeloPrevisao.previsao([45,0])

Grad:  [4.95692028e-08 2.23061412e-06 0.00000000e+00]
[0.62450018 1.09722455 0.28148128]


49.99960473517724

In [4331]:
modeloPrevisao.getParameters()

array([0.62450018, 1.09722455, 0.28148128])

In [8]:
EPSILON = 0.4
modeloPrevisao = PredicaoDoTempoNaDoca()
def calculeMelhorDoca(docas, hora):
    dice = np.random.random()
    nDocas = docas.getNumeroDocas()
    if dice < EPSILON:
        return np.random.choice(nDocas)

    menorTempo = float('inf')
    iDocaMenor = -1
    for i in range(nDocas):
        itemsList = docas.getNumeroItemsDeCadaCaminhao(i)
        tempoPrevisto = 0
        for items in itemsList:
            tempoPrevisto += modeloPrevisao.previsao([items,i])
        tempoPrevisto -= docas.getTempoDescarregandoNaDoca(i, hora)
        if tempoPrevisto < menorTempo:
            menorTempo = tempoPrevisto 
            iDocaMenor = i
    return iDocaMenor

def updateModel(historico):
    erroMedio = 0
    for caminhaoI in historico:
        x = [caminhaoI.items,caminhaoI.getDoca()]
        valorReal = caminhaoI.tempoTotalDescarregando()
        modeloPrevisao.atualizandoParametros(x,valorReal)
        erroMedio += abs(valorReal - modeloPrevisao.previsao(x))
        if VERBOSE:
            print(valorReal, x, 'erro:  ', 
                  abs(valorReal - modeloPrevisao.previsao(x)))
    return erroMedio/len(historico)


In [35]:
VERBOSE = False

for i in range(100):
    tempoMedio, docas = simuleUmDia()
    historico = docas.getHistorico()
    #updateModel(historico)
    print('erro medio: ', updateModel(historico))

erro medio:  0.09843532826890598
erro medio:  0.0997784070763231
erro medio:  0.06750113131693627
erro medio:  0.0693232551360019
erro medio:  0.1047348713199506
erro medio:  0.08392131460014837
erro medio:  0.08521131765708609
erro medio:  0.08334867807999231
erro medio:  0.06750426226983665
erro medio:  0.10881892774800539
erro medio:  0.10509844285951764
erro medio:  0.11727739196235157
erro medio:  0.1271534946285024
erro medio:  0.11276406379079298
erro medio:  0.08435853687148845
erro medio:  0.06819283060778461
erro medio:  0.0638255941810393
erro medio:  0.10215437352783645
erro medio:  0.07179010248390925
erro medio:  0.11209276436880114
erro medio:  0.10523689003808366
erro medio:  0.08353655361388039
erro medio:  0.09147311626715282
erro medio:  0.08693265102053764
erro medio:  0.07557940178982206
erro medio:  0.1137997108446937
erro medio:  0.0900207232863881
erro medio:  0.1274450728793082
erro medio:  0.10888649254681065
erro medio:  0.08486447438832319
erro medio:  0.093

In [37]:
modeloPrevisao.previsao([42,1])

42.82914701876894

0.47452938164644565

In [596]:
caminhaoI